## Demo sources plugin

This example demonstrates the usage of [sources plugins](../plugins/sources_plugin.rst) in earthkit-data.

We will use the **earthkit-data-demo-source** plugin, which allows accessing data from a SQL database. It has to be installed to make the exercise work. 

In [1]:
# !pip install --quiet git+https://github.com/ecmwf/earthkit-data-demo-source

First, we create a test database.

In [2]:
import earthkit.data

import os
import sqlite3

DATA = [
    (50, 3.3, "2001-01-01 00:00:00", 4.9),
    (51, -3, "2001-01-02 00:00:00", 7.3),
    (50.5, -1.8, "2001-01-03 00:00:00", 5.5),
]


def make_db():
    if os.path.exists("_test.db"):
        os.unlink("_test.db")

    conn = sqlite3.connect("_test.db")
    c = conn.cursor()
    c.execute(
        """CREATE TABLE data(
                    lat NUMBER,
                    lon NUMBER,
                    time TEXT,
                    value NUMBER)"""
    )
    c.executemany("INSERT INTO data VALUES(?,?,?,?);", DATA)
    conn.commit()

make_db()

The plugin implements a new earthkit-data source called "demo-source". We can simply use *earthkit.data.from_source()* to read our database.

In [3]:
ds = earthkit.data.from_source(
        "demo-source",
        "sqlite:///_test.db",
        "select * from data;",
        parse_dates=["time"],
    )
df = ds.to_pandas()
df

,lat,lon,time,value
0,50.0,3.3,2001-01-01,4.9
1,51.0,-3.0,2001-01-02,7.3
2,50.5,-1.8,2001-01-03,5.5
